# Data Collection

## Libraries

In [1]:
#Web Scraping
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time

#Data manage
import pandas as pd

## Data to collect

In [2]:
#Pages from where we collect the information

urls = ['https://onestop.fiu.edu/finances/estimate-your-costs/undergraduate-tuition-fees/',
'https://www.francis.edu/Undergraduate-Tuition-and-Fees/',
'https://www.jacobs-university.de/study/undergraduate/fees-finances',
'https://www.hud.ac.uk/international/tuition-fees/',
'https://www.centennialcollege.ca/studenthub/international/international-students/settling-in-canada/cost-of-living/']

In [3]:
universities = []
for i in urls:
    univ = i.split("/")[2]
    print(univ)
    universities.append(univ)

onestop.fiu.edu
www.francis.edu
www.jacobs-university.de
www.hud.ac.uk
www.centennialcollege.ca


## Data Collection

In [4]:
#Create an empty dataframe so we can use it to collect the data
empty_dicc = {}

database = pd.DataFrame(empty_dicc)
database.head()

""


## Web Scraper

In [5]:
#Related words to search for
woi = ['foo', 'heal', 'book', 'personal', 'accom', 'transp', 'rent', 'travel', 'meal', 'room']
cost_freq = ['week', 'annual', 'semester', 'month']

#Loop through the list to get the information

for url in range(len(urls)):
    
    #Open the browser with Selenium
    browser = webdriver.Firefox(executable_path='/home/bryan/Escritorio/Data Ch/geckodriver')
    browser.get(urls[url])
    time.sleep(3)
    
    #Get the html info
    page_s = browser.page_source
    soup = BeautifulSoup(page_s,'html.parser')
    
    #Create an empty list to store the particular information we want
    cost = []
    a = soup.find_all("tr")
    for i in range(len(a)):
        L = []
        if i not in cost:
            L.append(a[i].text+" ")

        cost.append(L)
    
    #There was a website that didn't manage tr tags to store the information, so this was an exception
    if len(cost)==0:
        
        #On this case we need to click plegable bar, so you can find some information
        try:
                browser.find_element_by_xpath('//*[@id="Kamjzmob"]').click()
                soup = BeautifulSoup(page_s,'html.parser')
                a = soup.find_all("tr")
                for i in range(len(a)):
                    L = []
                    if i not in cost:
                        L.append(a[i].text+" ")

                    cost.append(L)
        
        #This wasn't using any tr tag, so you can't use this to get information, it was stored on the paragraphs
        except:
        
                a = soup.find_all("p")
                for i in range(len(a)):
                    L = []
                    if i not in cost:
                        L.append(a[i].text+" ")

                    cost.append(L)
                pass
    
    #Process the information, using split method
    c_info = []
    for string in cost:
        for ch in string:
            if '$' in ch:
                str_ch = ch.split('$')
                c_info.append(str_ch)
                
            elif '€' in ch:
                str_ch = ch.split('€')
                c_info.append(str_ch)
            
            elif '£' in ch:
                str_ch = ch.split('£')
                c_info.append(str_ch)
    
    #Now loop through the information collected to get what we are interested in      
    L_info = []
    L_fees = []
    numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    for info in c_info:
        for number in numbers:
            try:
                if (number in info[1]) and (info[0] not in L_fees):
                    L_fees.append(info[1])
                    L_info.append(info[0])
            except:
                pass
        
    t_inf = []
    t_cost = []
    u = []
    
    for i in range(len(L_info)):
        for word in woi:
            if (word in L_info[i].lower()) and (word not in t_inf):
                t_inf.append(word.lower())
                t_cost.append(L_fees[i])
                u.append(universities[url])
               
    #Store the information into the empty dataframe that we created
    d = {'Info':t_inf, 'Cost Value':t_cost, 'University':u}
    df = pd.DataFrame(d)
    
    database = pd.concat([database, df])
    
    #Stop using the browser
    browser.quit()
    time.sleep(3)
    print(t_inf)

['book', 'transp', 'room', 'personal']
['room', 'meal', 'heal']
['room']
['heal', 'accom', 'foo', 'travel']
['rent', 'foo', 'transp', 'personal']


In [6]:
database = database.drop_duplicates()
database

,Info,Cost Value,University
0,book,675,onestop.fiu.edu
1,transp,"1,147",onestop.fiu.edu
2,room,"3,744",onestop.fiu.edu
3,personal,"1,356",onestop.fiu.edu
0,room,"6,148n/a",www.francis.edu
1,meal,100 flex dollars and 8 guest meals),www.francis.edu
2,heal,"1,923",www.francis.edu
0,room,"20,000Room & Board*",www.jacobs-university.de
0,heal,"15,000 (scholarships available)",www.hud.ac.uk
1,accom,70.00,www.hud.ac.uk


## Clean Dataframe

In [7]:
num = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
cost_t = []
val = ""
for value in database['Cost Value']:
    for char in value:
        if char in num or (char == '.' or char ==','):
            val +=char
    cost_t.append(val)
    val = ""

In [8]:
info_t = []
for value in database['Info']:
    if value =='room' or value == 'rent' or value == 'accom':
        info_t.append('Accommodation costs')
    elif value =='foo' or value =='meal':
        info_t.append('Food costs')
    elif value =='personal':
        info_t.append('Personal Expenses')
    elif value =='heal':
        info_t.append('Health Insurance')
    elif value =='transp' or value =='travel':
        info_t.append('Transport costs')
    elif value =='book':
        info_t.append('Books and supplies')

In [9]:
database['Info'] = info_t
database['Cost Value'] = cost_t

In [10]:
database = database.reset_index().iloc[:,1:]
database

,Info,Cost Value,University
0,Books and supplies,675,onestop.fiu.edu
1,Transport costs,"1,147",onestop.fiu.edu
2,Accommodation costs,"3,744",onestop.fiu.edu
3,Personal Expenses,"1,356",onestop.fiu.edu
4,Accommodation costs,"6,148",www.francis.edu
5,Food costs,1008,www.francis.edu
6,Health Insurance,"1,923",www.francis.edu
7,Accommodation costs,"20,000",www.jacobs-university.de
8,Health Insurance,"15,000",www.hud.ac.uk
9,Accommodation costs,70.00,www.hud.ac.uk


In [11]:
database.to_csv('db.csv')

## Summary

As you can see, i can handle most of the information that was required, but as web pages don't follow any general structure, managing unstructured data is difficult, so making a general code for this case was quite difficult.

For a better performance you can use NLP to try to get the topics of interest and later get the information after the correct pre-processing data